# Data Validation — `fct_order_items` (NTU SCTP Module 2)

This notebook is a **data quality validation companion** for the project. It focuses on validating the integrity of the exported fact-level dataset (and optionally validating directly in BigQuery).

## Goals
- Confirm the dataset is non-empty and well-formed
- Validate key columns (types, missing values)
- Check for duplicates at the expected grain
- Sanity-check revenue and orders
- Produce a short validation summary suitable for the report/presentation

> Default mode: validate from CSV export. Optional BigQuery section is provided at the end.


## 0) Setup

Install dependencies (if needed):

```bash
pip install pandas numpy
```


In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)


## 1) Load data (CSV export)

Update `CSV_PATH` to your exported file.


In [3]:
# ✅ Change this path if needed
CSV_PATH = "results-20251216-151001 - results-20251216-151001.csv"

df = pd.read_csv(CSV_PATH)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'results-20251216-151001 - results-20251216-151001.csv'

## 2) Basic checks: shape, columns, nulls


In [ ]:
print("Rows:", len(df))
print("Columns:", len(df.columns))
print("\nColumn names:")
display(df.columns.to_list()) if 'display' in globals() else print(df.columns.to_list())

# Empty dataset check
if df.empty:
    raise ValueError("Dataset is empty. Export may have failed or query returned no rows.")

# Missing values summary
nulls = df.isna().sum().sort_values(ascending=False)
nulls[nulls > 0].head(20)

## 3) Schema validation (expected columns)

### Expected minimum columns
- `month`
- `category_name`
- `total_orders`
- `product_revenue`


In [ ]:
required_cols = {"month", "category_name", "total_orders", "product_revenue"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

print("✅ Required columns present.")

## 4) Type checks and normalization

We normalize:
- `month` → `YYYY-MM`
- numeric fields → numeric


In [ ]:
df["month"] = df["month"].astype(str).str.slice(0, 7)

for col in ["total_orders", "product_revenue"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

bad_numeric = df[["total_orders", "product_revenue"]].isna().sum()
print("Coerced-to-NaN counts (should be 0 or very low):")
print(bad_numeric)

# Fill NaNs after reporting
df[["total_orders", "product_revenue"]] = df[["total_orders", "product_revenue"]].fillna(0)

print("\nDate range:", df["month"].min(), "to", df["month"].max())
print("Unique categories:", df["category_name"].nunique())

## 5) Duplicate checks (at the exported grain)

Your export appears to be **monthly revenue by category**, so the expected grain is:
- one row per (`month`, `category_name`)

This section checks that the export respects that grain.


In [ ]:
dup_count = df.duplicated(subset=["month", "category_name"]).sum()
print("Duplicate rows at (month, category_name) grain:", dup_count)

if dup_count > 0:
    print("\nSample duplicates:")
    dups = df[df.duplicated(subset=["month", "category_name"], keep=False)].sort_values(["month","category_name"])
    dups.head(20)

## 6) Sanity checks: non-negative metrics, top months, top categories


In [ ]:
neg_orders = (df["total_orders"] < 0).sum()
neg_rev = (df["product_revenue"] < 0).sum()
print("Negative orders rows:", neg_orders)
print("Negative revenue rows:", neg_rev)

monthly = df.groupby("month", as_index=False).agg(total_orders=("total_orders","sum"), product_revenue=("product_revenue","sum")).sort_values("month")
top_month = monthly.loc[monthly["product_revenue"].idxmax()]
print("\nPeak revenue month:")
print(top_month)

cat_totals = df.groupby("category_name", as_index=False).agg(product_revenue=("product_revenue","sum"), total_orders=("total_orders","sum")).sort_values("product_revenue", ascending=False)
print("\nTop 10 categories by revenue:")
cat_totals.head(10)

## 7) Validation summary (copy/paste for report)

This prints a short summary suitable for your documentation / slides.


In [ ]:
total_rev = float(df["product_revenue"].sum())
top5_share = cat_totals.head(5)["product_revenue"].sum() / total_rev if total_rev else 0

summary = {
    "rows": int(len(df)),
    "date_range": f"{df['month'].min()} to {df['month'].max()}",
    "unique_categories": int(df['category_name'].nunique()),
    "duplicates_at_expected_grain": int(dup_count),
    "negative_orders_rows": int(neg_orders),
    "negative_revenue_rows": int(neg_rev),
    "peak_revenue_month": str(top_month['month']),
    "top5_category_revenue_share_pct": round(float(top5_share*100), 1)
}

summary

In [ ]:
print("Validation Summary:")
print(f"- Rows validated: {summary['rows']}")
print(f"- Date range: {summary['date_range']}")
print(f"- Unique categories: {summary['unique_categories']}")
print(f"- Duplicates at (month, category_name) grain: {summary['duplicates_at_expected_grain']}")
print(f"- Negative orders rows: {summary['negative_orders_rows']}")
print(f"- Negative revenue rows: {summary['negative_revenue_rows']}")
print(f"- Peak revenue month observed: {summary['peak_revenue_month']}")
print(f"- Revenue concentration: Top 5 categories ≈ {summary['top5_category_revenue_share_pct']}% of total revenue")

## (Optional) Direct validation from BigQuery

Use this only if you want to validate the **actual fact table** in the warehouse.

Prereqs:
- `pip install google-cloud-bigquery db-dtypes`
- Authentication set up (ADC or service account)


In [ ]:
# Uncomment to use BigQuery
# from google.cloud import bigquery
#
# PROJECT_ID = "your-gcp-project-id"
# DATASET = "olist_analytics"
# FACT_TABLE = "fct_order_items"
#
# client = bigquery.Client(project=PROJECT_ID)
#
# # Example checks
# queries = {
#     "row_count": f"SELECT COUNT(*) AS n FROM `{PROJECT_ID}.{DATASET}.{FACT_TABLE}`",
#     "null_checks": f"""
#         SELECT
#           SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS null_order_id,
#           SUM(CASE WHEN product_id IS NULL THEN 1 ELSE 0 END) AS null_product_id,
#           SUM(CASE WHEN price IS NULL THEN 1 ELSE 0 END) AS null_price
#         FROM `{PROJECT_ID}.{DATASET}.{FACT_TABLE}`
#     """,
#     "duplicate_grain": f"""
#         SELECT COUNT(*) AS dup_rows
#         FROM (
#           SELECT order_id, order_item_id, COUNT(*) c
#           FROM `{PROJECT_ID}.{DATASET}.{FACT_TABLE}`
#           GROUP BY 1,2
#           HAVING c > 1
#         )
#     """
# }
#
# for name, q in queries.items():
#     print("\n---", name, "---")
#     display(client.query(q).to_dataframe())
